In [ ]:

# from crewai import Agent, Task, Crew
# from crewai_tools import SerperDevTool
# from dotenv import load_dotenv
# import os

# # Load environment variables
# load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")  # Add your Serper API key to .env

# # Initialize LLM and Tool
# llm = "gpt-4-0125-preview"
# search_tool = SerperDevTool()  # Web search tool for latest data (previously not using)

# companies = input("Enter companies to analyze (comma-separated, e.g., Apple, Tesla): ").split(", ")
# companies_str = ", ".join(companies)

# # Define Agents with company-specific focus and search tool
# research_agent = Agent(
#     role='Company Researcher',
#     goal=f'Gather financial data and market position for {companies_str} up the latest available date and time currently',
#     backstory='You are an expert in company-specific financial research with access to latest available date and time web data',
#     verbose=True,
#     llm=llm,
#     tools=[search_tool]  # Add web search capability
# )

# analysis_agent = Agent(
#     role='Company Analyst',
#     goal=f'Analyze investment potential for {companies_str} based on latest data',
#     backstory='You specialize in evaluating company investment opportunities',
#     verbose=True,
#     llm=llm,
#     tools=[search_tool]  # Add web search for analysis
# )

# report_agent = Agent(
#     role='Investment Advisor',
#     goal=f'Create investment recommendations for {companies_str} with current insights',
#     backstory='You are a financial advisor creating tailored investment reports',
#     verbose=True,
#     llm=llm
# )

# # Define Tasks with company focus
# research_task = Task(
#     description=f'Research current financial performance, market trends, and news for {companies_str} up to latest as of today date and time using web search',
#     agent=research_agent,
#     expected_output=f'A summary of financial data and market position for {companies_str} based on latest available data'
# )

# analysis_task = Task(
#     description=f'Analyze the investment potential of {companies_str} including risk assessment using current data',
#     agent=analysis_agent,
#     expected_output=f'Detailed investment analysis for {companies_str} with risk levels based on latest trends'
# )

# report_task = Task(
#     description=f'Create an investment recommendation report for {companies_str} based on the latest analysis',
#     agent=report_agent,
#     expected_output=f'A structured report with investment recommendations for {companies_str}'
# )

# # Cell 6: Create and Run the Crew
# investment_crew = Crew(
#     agents=[research_agent, analysis_agent, report_agent],
#     tasks=[research_task, analysis_task, report_task],
#     verbose=True
# )

# # Execute the crew
# result = investment_crew.kickoff()

# # Cell 7: Display and Save Results
# print(f"Investment Report for {companies_str} (April 2025):")
# print(result)

# # Save with company-specific filename
# filename = f"investment_report_{'_'.join(companies)}_2025.txt"
# with open(filename, 'w') as f:
#     f.write(str(result))
# print(f"Report saved to '{filename}'")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ad388bce-9ffa-45e0-818c-171938569b95                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 39b2e12e-8ca8-4bcb-abcb-fee5916d969b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 39b2e12e-8ca8-4bcb-abcb-fee5916d969b
       Status: Executing Task...
    └── 🤖 Agent: Company Researcher
            Status: In Progress

# Agent: Company Researcher
## Task: Research current financial performance, market trends, and news for DBS BANK up to latest as of today date and time using web search


🚀 Crew: crew
└── 📋 Task: 39b2e12e-8ca8-4bcb-abcb-fee5916d969b
       Status: Executing Task...
    └── 🤖 Agent: Company Researcher
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 39b2e12e-8ca8-4bcb-abcb-fee5916d969b
│      Assigned to: Company Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Researcher
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
├── 📋 Task: 1e536ab6-9500-4d80-82a1-2205f6d58ea0
│      Assigned to: Company Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (7)
└── 📋 Task: 978d44f0-ee3e-4f87-87bc-bf1deb32af1d
       Status: Executing Task...
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 39b2e12e-8ca8-4bcb-abcb-fee5916d969b
│      Assigned to: Company Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Researcher
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
├── 📋 Task: 1e536ab6-9500-4d80-82a1-2205f6d58ea0
│      Assigned to: Company Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (7)
└── 📋 Task: 978d44f0-ee3e-4f87-87bc-bf1deb32af1d
       Assigned to: Investment Advisor
       Status: ✅ Completed
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 978d44f0-ee3e-4f87-87bc-bf1deb32af1d                                                                     │
│  Agent: Investment Advisor                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ad388bce-9ffa-45e0-818c-171938569b95                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Investment Report for DBS BANK (April 2025):
**Investment Recommendation Report for DBS Bank**

**Executive Summary:**
DBS Bank has exhibited remarkable financial performance in 2023 with significant growth in total income and net profit. The bank's strategic positioning in key Asian markets and its strong performance in wealth management position it as a leading investment opportunity. However, potential investors should consider the risks associated with recent leadership transitions, the bank's ongoing digital transformations, and broader economic uncertainties.

**Financial Performance and Outlook:**
DBS Bank's financial health is strong, with a 22% increase in total income and a 26% rise in net profit in 2023. The wealth management division has shown exceptional performance, contributing to an increasingly diversified revenue stream. Forward-looking statements from the bank project sustained growth, particularly in net interest and noninterest income, supporting a positive investm

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
import os
import datetime

# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

# Initialize LLM and Tool
llm = "gpt-4-0125-preview"
search_tool = SerperDevTool()

companies = input("Enter companies to analyze (comma-separated, e.g., Apple, Tesla): ").split(", ")
companies_str = ", ".join(companies)

# Get the current date and time
current_time = datetime.datetime.now()
filename_datetime = current_time.strftime("%Y/%m/%d/%H")

# Define Agents with company-specific focus and search tool, including current time
research_agent = Agent(
    role='Company Researcher',
    goal=f'Gather financial data and market position for {companies_str} up to the latest available date and time, specifically {current_time}.',
    backstory='You are an expert in company-specific financial research with access to latest available date and time web data.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

analysis_agent = Agent(
    role='Company Analyst',
    goal=f'Analyze investment potential for {companies_str} based on the latest data available up to {current_time}.',
    backstory='You specialize in evaluating company investment opportunities.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

report_agent = Agent(
    role='Investment Advisor',
    goal=f'Create investment recommendations for {companies_str} with current insights up to {current_time}.',
    backstory='You are a financial advisor creating tailored investment reports.',
    verbose=True,
    llm=llm
)

# Define Tasks with company focus and current time
research_task = Task(
    description=f'Research current financial performance, market trends, and news for {companies_str} up to {current_time} using web search.',
    agent=research_agent,
    expected_output=f'A summary of financial data and market position for {companies_str} based on data up to {current_time}.'
)

analysis_task = Task(
    description=f'Analyze the investment potential of {companies_str} including risk assessment using current data up to {current_time}.',
    agent=analysis_agent,
    expected_output=f'Detailed investment analysis for {companies_str} with risk levels based on trends up to {current_time}.'
)

report_task = Task(
    description=f'Create an investment recommendation report for {companies_str} based on the latest analysis up to {current_time}.',
    agent=report_agent,
    expected_output=f'A structured report with investment recommendations for {companies_str} based on data up to {current_time}.'
)

# Create and Run the Crew
investment_crew = Crew(
    agents=[research_agent, analysis_agent, report_agent],
    tasks=[research_task, analysis_task, report_task],
    verbose=True
)

# Execute the crew
result = investment_crew.kickoff()

# Display and Save Results
print(f"Investment Report for {companies_str} ({filename_datetime}):")
print(result)

# Save with company-specific filename
filename = f"investment_report_{'_'.join(companies)}_{filename_datetime.replace('/', '_')}.txt"
with open(filename, 'w') as f:
    f.write(str(result))
print(f"Report saved to '{filename}'")

AttributeError: 'str' object has no attribute 'strftime'